In [2]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

In [3]:
book_List = pd.read_excel("C:/프로젝트/k-d/2-Project/주제/data/998.인기대출_교보_Yes24 추가/End_data_Test3.xlsx")
# book_List = pd.read_csv("C:/k-디지털 프로젝트/2-Project/데이터/최종 백데이터/End_data.csv")
book_List.head(1)

,ISBN_THIRTEEN_NO,BOOK_TITLE_NM,AUTHR_NM,BOOK_INTRCN_CN,PBLICTE_DE,AGE_FLAG_NM,SEXDSTN_FLAG_NM,KDC_NM,Yes24_Price,Yes24_Score,Yes24_Review
0,9791130620893,돈 공부는 처음이라 - 0원부터 시작하는 난생처음 부자 수업,"김종봉, 제갈현열 (지은이)","사람들이 잘못 알고 있는 돈에 대한 상식을 짚어주고, 잘못된 상식으로 지금까지 돈 ...",2019,20대,여성,NaN,-1.0,9.22,8673


In [4]:
book_List.shape

(22094, 11)

In [5]:
# '나이'를 기반으로 'Age_Code' 컬럼 생성
def map_age_code(AGE_FLAG_NM):
    if AGE_FLAG_NM in ['영유아', '유아']:
        return 1
    elif AGE_FLAG_NM == '초등':
        return 2
    elif AGE_FLAG_NM == '청소년':
        return 3
    elif AGE_FLAG_NM == '20대':
        return 4
    elif AGE_FLAG_NM == '30대':
        return 5
    elif AGE_FLAG_NM == '40대':
        return 6
    elif AGE_FLAG_NM == '50대':
        return 7
    elif AGE_FLAG_NM == '60대이상':
        return 8
    else:
        return None  # 예외 처리: 다른 값이 들어오면 None 반환

# 'Age_Code' 컬럼 추가
book_List['Age_Code'] = book_List['AGE_FLAG_NM'].apply(map_age_code)


# '성별'를 기반으로 'Sex_Code' 컬럼 생성
def map_sex_code(SEXDSTN_FLAG_NM):
    if SEXDSTN_FLAG_NM == '남성':
        return 1
    elif SEXDSTN_FLAG_NM == '여성':
        return 2
    else:
        return None  # 예외 처리: 다른 값이 들어오면 None 반환

# 'Age_Code' 컬럼 추가
book_List['Sex_Code'] = book_List['SEXDSTN_FLAG_NM'].apply(map_sex_code)


In [6]:
book_List.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22094 entries, 0 to 22093
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ISBN_THIRTEEN_NO  22094 non-null  object 
 1   BOOK_TITLE_NM     22083 non-null  object 
 2   AUTHR_NM          22027 non-null  object 
 3   BOOK_INTRCN_CN    20490 non-null  object 
 4   PBLICTE_DE        21991 non-null  object 
 5   AGE_FLAG_NM       22094 non-null  object 
 6   SEXDSTN_FLAG_NM   22094 non-null  object 
 7   KDC_NM            14128 non-null  float64
 8   Yes24_Price       22094 non-null  float64
 9   Yes24_Score       22094 non-null  float64
 10  Yes24_Review      22094 non-null  int64  
 11  Age_Code          7735 non-null   float64
 12  Sex_Code          22094 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 2.2+ MB


In [7]:
# 'Yes24_Price', 'Yes24_Score', 'Yes24_Review' 컬럼 이름 변경
book_List.rename(columns={'Yes24_Price': 'avg_price', 'Yes24_Score': 'avg_Score', 'Yes24_Review': 'Review_Sum'}, inplace=True)

In [8]:
# 주요 컬럼으로 데이터 프레임 생성
# New column = Age_Code, avg_price, avg_Score, Sale_Sum
col_lst = ['ISBN_THIRTEEN_NO','BOOK_TITLE_NM','AUTHR_NM','Sex_Code','BOOK_INTRCN_CN','PBLICTE_DE','KDC_NM','avg_price','avg_Score','Review_Sum']
book_List_df = book_List[col_lst]

book_List_df.head(1)

,ISBN_THIRTEEN_NO,BOOK_TITLE_NM,AUTHR_NM,Sex_Code,BOOK_INTRCN_CN,PBLICTE_DE,KDC_NM,avg_price,avg_Score,Review_Sum
0,9791130620893,돈 공부는 처음이라 - 0원부터 시작하는 난생처음 부자 수업,"김종봉, 제갈현열 (지은이)",2,"사람들이 잘못 알고 있는 돈에 대한 상식을 짚어주고, 잘못된 상식으로 지금까지 돈 ...",2019,NaN,-1.0,9.22,8673


In [9]:
# 사용할 컬럼 재정의 후 중복제거  연령별 제거 후 실행해보기 데이터가 너무 많음... 23만개  =  연령별 포함 groupby =  30만개
book_List_df = book_List_df[['ISBN_THIRTEEN_NO','BOOK_TITLE_NM','AUTHR_NM','Sex_Code','BOOK_INTRCN_CN','PBLICTE_DE','KDC_NM','avg_price','avg_Score','Review_Sum']].drop_duplicates()

# 사용할 컬럼 재정의 후 중복제거
# book_List_df = book_List_df[['ISBN_THIRTEEN_NO','BOOK_TITLE_NM','AUTHR_NM','Age_Code','Sex_Code','BOOK_INTRCN_CN','PBLICTE_DE','KDC_NM','avg_price','avg_Score','Review_Sum']].drop_duplicates()

In [10]:
book_List_df.shape

(22094, 10)

In [25]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from konlpy.tag import Okt
import re

# 장르별
book_List_df['genres_literal'] = book_List_df['KDC_NM'].astype(str)

# 책 재목
book_List_df['title_literal'] = book_List_df['BOOK_TITLE_NM'].astype(str)

# 책 줄거리 내용 
# book_List_df_Avg['book_intrcn_literal'] = book_List_df_Avg['BOOK_INTRCN_CN'].astype(str)
# 이 코드는 주어진 텍스트 데이터에 대해 불용어 제거, 특수 문자 제거, 형태소 분석 등을 수행하여 텍스트 데이터를 정제하는 과정 포함
# 한글 형태소 분석기 초기화
okt = Okt()

def preprocess_text(text):
    if pd.isnull(text):
        return ''  # Return empty string for NaN values
    else:
        # 불용어 제거
        stop_words = ['는', '은', '이', '가', '을', '를', '의', '에서', '에게', '에', '으로', '로']
        words = [word for word in okt.morphs(text) if word not in stop_words]

        # 특수 문자 제거
        words = [re.sub(r'[^a-zA-Z가-힣0-9]', ' ', word) for word in words]

        # 형태소 분석
        words = [okt.morphs(word) for word in words]

        # 공백 제거 및 단어 재조합
        words = [' '.join(word) for word in words]

        return ' '.join(words)

# 텍스트 데이터에 전처리 적용
book_List_df['book_intrcn_literal'] = book_List_df['BOOK_INTRCN_CN'].apply(preprocess_text)

# Concatenate 'title_literal' with 'genres_literal' and 'book_intrcn_literal'
book_List_df['combined_features'] = (
    book_List_df['genres_literal'] + ' ' +
    book_List_df['book_intrcn_literal'] + ' ' +
    book_List_df['title_literal']
)

# CountVectorizer를 이용한 장르 유사도 계산
count_vect = CountVectorizer(min_df=1, ngram_range=(1, 2))
genre_mat_dense = count_vect.fit_transform(book_List_df['combined_features'])

# Convert dense matrix to sparse matrix
genre_mat = csr_matrix(genre_mat_dense)

# 코사인 유사도 계산 (희소 행렬 직접 사용)
genre_sim = cosine_similarity(genre_mat, dense_output=False)

# 선택한 책에 대한 유사도만을 추출할 책의 인덱스 설정
title_index = 0

# 선택한 책 대한 유사도만을 추출
selected_book_similarity = genre_sim[title_index, :].reshape(-1, 1)

# Choose a lower dimensionality
n_components = 30

# Apply TruncatedSVD to reduce dimensionality
svd = TruncatedSVD(n_components=n_components)
genre_mat_reduced = svd.fit_transform(genre_mat)

# Compute cosine similarity on the reduced matrix
genre_sim_reduced = cosine_similarity(genre_mat_reduced, dense_output=False)

# 선택한 책에 대한 유사도만을 추출
selected_book_similarity_reduced = genre_sim_reduced[title_index, :].reshape(-1, 1)

In [26]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def calculate_intrcn_similarity(df):
    df['BOOK_INTRCN_CN'].fillna('', inplace=True)
    count_vectorizer = CountVectorizer(stop_words='english')
    intrcn_matrix = count_vectorizer.fit_transform(df['BOOK_INTRCN_CN'])
    intrcn_sim = cosine_similarity(intrcn_matrix, intrcn_matrix)
    return intrcn_sim

def normalize_feature(feature):
    return (feature - feature.min()) / (feature.max() - feature.min())

def find_sim_movie(df, sim_matrix, title_isbn, top_n):
    title_book = df[df['ISBN_THIRTEEN_NO'] == title_isbn]

    if title_book.empty:
        print(f"No book found with ISBN {title_isbn}")
        return pd.DataFrame()

    title_index = title_book.index.values[0]
    
    df["similarity"] = sim_matrix[title_index].toarray().flatten()

    df = df[df["avg_Score"] != -1]

    df["normalized_score"] = normalize_feature(df["avg_Score"])
    df["normalized_price"] = 1 - normalize_feature(df["avg_price"])
    df["normalized_Sale"] = normalize_feature(df["Review_Sum"])

    intrcn_sim = calculate_intrcn_similarity(df)
    df["intrcn_similarity"] = intrcn_sim[title_index]

    df["combined_similarity"] = (
        df["similarity"] +
        0.1 * df["normalized_score"] + # 별점
        0.1 * df["normalized_price"] + # 가격
        0.3 * df["normalized_Sale"] +  # 리뷰 수
        0.5 * df["intrcn_similarity"] # 제목, 장르, 책줄거리
    )

    temp = df.sort_values(by=["combined_similarity", "Review_Sum"], ascending=[False, False])
    temp = temp.drop_duplicates(subset=['ISBN_THIRTEEN_NO','BOOK_TITLE_NM','Sex_Code','AUTHR_NM','BOOK_INTRCN_CN','PBLICTE_DE','KDC_NM','Review_Sum'])

    # 해당 문자열 자르기는 추출된 값을 보기 좋게하기위해 작성된 코드 
    # 지워도 무관함
    temp['BOOK_TITLE_NM'] = temp['BOOK_TITLE_NM'].str[:10]
    temp['AUTHR_NM'] = temp['AUTHR_NM'].str[:10]
    temp['BOOK_INTRCN_CN'] = temp['BOOK_INTRCN_CN'].str[:10]

    final_index = temp.index.values[:min(top_n, len(temp))]

    return temp.iloc[:top_n] if len(temp) >= top_n else temp

similar_books = find_sim_movie(book_List_df.copy(), genre_sim, 9791188874262, 16)
# 9788933870792 	설민석의 삼국지 1 - 누구나 쉽게 시작하고, 모두가 빠져드는 이야기

In [35]:
from tabulate import tabulate

# 중복된 행 제거
unique_similar_books = similar_books.drop_duplicates(subset=['ISBN_THIRTEEN_NO','BOOK_TITLE_NM','Sex_Code','AUTHR_NM','KDC_NM','avg_price','avg_Score','Review_Sum','intrcn_similarity'])

# avg_price가 -1일 경우 "품절"이라는 텍스트 출력
def format_avg_price(row):
    if row['avg_price'] == -1:
        return "품절"
    else:
        return str(row['avg_price'])

# 품절 여부에 따라 avg_price 포맷팅
unique_similar_books['formatted_avg_price'] = unique_similar_books.apply(format_avg_price, axis=1)

print("\n 남성에게 추천하는 책:")
print(tabulate(unique_similar_books[unique_similar_books['Sex_Code'] == 1][['ISBN_THIRTEEN_NO','BOOK_TITLE_NM','Sex_Code','AUTHR_NM','KDC_NM','avg_price','avg_Score','Review_Sum','intrcn_similarity']],
               headers='keys', tablefmt='psql', showindex=False, numalign="center", stralign="center", colalign=("center",)*7))

print("\n 여성에게 추천하는 책:")
print(tabulate(unique_similar_books[unique_similar_books['Sex_Code'] == 2][['ISBN_THIRTEEN_NO','BOOK_TITLE_NM','Sex_Code','AUTHR_NM','KDC_NM','avg_price','avg_Score','Review_Sum','intrcn_similarity']],
               headers='keys', tablefmt='psql', showindex=False, numalign="center", stralign="center", colalign=("center",)*7))



 남성에게 추천하는 책:
+--------------------+--------------------+------------+--------------------+----------+-------------+-------------+--------------+---------------------+
|  ISBN_THIRTEEN_NO  |   BOOK_TITLE_NM    |  Sex_Code  |      AUTHR_NM      |  KDC_NM  |  avg_price  |  avg_Score  |  Review_Sum  |  intrcn_similarity  |
|--------------------+--------------------+------------+--------------------+----------+-------------+-------------+--------------+---------------------|
|   9791188874262    | 설민석의 한국사 대 |     1      | 설민석, 스토리박스 |   nan    |     -1      |    9.81     |    17970     |          1          |
|   9791188874217    | (설민석의) 한국사  |     1      |  글: 설민석,그림:  |   911    |     -1      |    9.77     |    16020     |      0.602216       |
|   9791188874460    | 설민석의 한국사 대 |     1      | 설민석, 스토리박스 |   nan    |     -1      |    9.79     |    10050     |      0.515388       |
|   9791196232160    | (설민석의) 한국사  |     1      |  글: 설민석,그림:  |   911    |   4724.5    |    9.89     |   